<a href="https://colab.research.google.com/github/sakamoto-hands-on/Python_InteractiveComputing_and_Visualization/blob/master/Profiling_Code_and_Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Profiling the code with cProfile

In [0]:
n = 100000

### Python標準の「sum」とNumPyを比較

In [2]:
%timeit sum([1./i**2 for i in range(1,n)])

10 loops, best of 3: 32.6 ms per loop


In [3]:
%%timeit s=0.
for i in range(1,n):
  s+=1./i**2

10 loops, best of 3: 30.5 ms per loop


In [0]:
import numpy as np

In [5]:
%timeit np.sum(1./np.arange(1.,n)**2)

The slowest run took 7.64 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 770 µs per loop


わたしが参照しているテキストでは137倍の速度が出ていると書かれている。Colaboratory上ではおよそ50倍かな？



---



### %prun magic commandを使って、詳細なレポートを作成

In [0]:
def step(*shape):
  return 2*(np.random.random_sample(shape)<.5)-1

In [7]:
%%prun -s cumulative -q -l 10 -T prun0
n=10000
iterations=50
x=np.cumsum(step(iterations,n),axis=0)
bins=np.arange(-30,30,1)
y=np.vstack([np.histogram(x[i,:],bins)[0] for i in range(iterations)])

 
*** Profile printout saved to text file 'prun0'. 


In [8]:
print(open('prun0','r').read())

         2829 function calls (2527 primitive calls) in 0.041 seconds

   Ordered by: cumulative time
   List reduced from 58 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.041    0.041 {built-in method builtins.exec}
        1    0.000    0.000    0.040    0.040 <string>:2(<module>)
   354/52    0.000    0.000    0.022    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.007    0.007    0.018    0.018 <ipython-input-6-1ea381f77015>:1(step)
        1    0.000    0.000    0.018    0.018 <string>:6(<listcomp>)
       50    0.000    0.000    0.018    0.000 <__array_function__ internals>:2(histogram)
       50    0.001    0.000    0.018    0.000 histograms.py:680(histogram)
       50    0.000    0.000    0.015    0.000 <__array_function__ internals>:2(sort)
       50    0.000    0.000    0.015    0.000 fromnumeric.py:826(sort)
       50    0.014    0.000    0.0



---



# Profiling the code line by line

In [9]:
pip install line-profiler

     |████████████████████████████████| 92kB 4.5MB/s 
  Created wheel for line-profiler: filename=line_profiler-2.1.2-cp36-cp36m-linux_x86_64.whl size=180886 sha256=148d33829455d8d4c2d4158ba14da25cd24263182425bda71027c051022b99b1
  Stored in directory: /root/.cache/pip/wheels/05/7d/9b/aafbe8d78dc2b2c644d2efd2f060ab3258143860142575193a
Successfully built line-profiler


In [0]:
import numpy as np
%load_ext line_profiler

In [11]:
%%writefile simulation01.py
import numpy as np

def step(*shape):
  return 2*(np.random.random_sample(shape)<.5)-1

def simulate(iterations,n=10000):
  s=step(iterations,n)
  x=np.cumsum(s,axis=0)
  bins=np.arange(-30,30,1)
  y=np.vstack([np.histogram(x[i,:],bins)[0] for i in range(iterations)])
  return y

Writing simulation01.py


In [12]:
%ls

prun0  sample_data/  simulation01.py


In [0]:
from simulation01 import simulate

In [14]:
%lprun -T lprof0 -f simulate simulate(50)


*** Profile printout saved to text file 'lprof0'. 




---



# Profiling memory usage

In [16]:
pip install memory_profiler

     |████████████████████████████████| 40kB 3.3MB/s 
  Created wheel for memory-profiler: filename=memory_profiler-0.55.0-cp36-none-any.whl size=27174 sha256=08b85808527eb177ae0d0236a0093d737ad7c90ec94f3352a5616553582838c1
  Stored in directory: /root/.cache/pip/wheels/f0/ff/63/fdbff3f1e1b76ad4eae491dd5b190902906b093e93eb86dd5a
Successfully built memory-profiler


In [0]:
%load_ext memory_profiler

In [18]:
%%writefile memscript.py
def my_func():
  a=[1]*1000000
  b=[2]*9000000
  del b
  return a

Writing memscript.py


In [19]:
from memscript import my_func
%mprun -T mprof0 -f my_func my_func()



*** Profile printout saved to text file mprof0. 
